In [1]:
import arrow
import dmyplant2 as dmp2
import pandas as pd
import logging
import sys

logfilename = 'FSM.log'
logging.basicConfig(
    filename=logfilename,
    filemode='w',
    format='%(asctime)s %(levelname)s %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.ERROR
)

try:
    dmp2.cred()
    mp = dmp2.MyPlant(0)
except Exception as err:
    print(str(err))
    sys.exit(1)

In [17]:
lookup = '' # '°C'
def sfun(x):
    return (
        (str(lookup).upper() in str(x['id']).upper()) or \
        (str(lookup).upper() in str(x['name']).upper()) or \
        (str(lookup).upper() in str(x['unit']).upper()) or \
        ((str(lookup).upper() in str(x['myPlantName']).upper())))
df = dmp2.MyPlant.get_dataitems()
df = df[df.apply(lambda x: sfun(x), axis=1)].reset_index()
lookup = 'Para_'
df = df[~df.apply(lambda x: sfun(x), axis=1)].reset_index()
df = df.sort_values('name')
df

,level_0,index,id,name,unit,myPlantName
781,2463,2463,1000083,AFRUL_RUL,oph,Air Filter Remaining Life
159,478,478,10009029,ATD1,NaN,1. ATD information for the table
2204,6799,6799,10009028,ATD2,NaN,2. ATD information for the table
1641,5104,5104,15785,ActiveState,NaN,Active State
1630,5058,5058,127,Aux_CH4Cont,%,CH4 content
...,...,...,...,...,...,...
1241,3901,3901,1040057,valverecession_riv_cyl24_deltaVRSinceInit,mm,Valve Recession - Cyl24 - Right Inlet Wear
743,2363,2363,1039961,valverecession_riv_cyl24_init,mm,Valve Recession - Cyl24 - Right Inlet - Init V...
1318,4142,4142,10051115,ventilation_rprdiff_er,,Room difference pressure
851,2676,2676,10050740,vibration_measurement_engine_1,mm/s,Pos. 1 Engine vibration measurement


In [18]:
dl = df[['id','name','unit','myPlantName']].to_dict('records')
dl;

In [19]:
e = dmp2.Engine.from_sn(mp, '1486152')
e.dash

{'Name': 'Forsa Hartmoor M02',
 'Engine ID': 'M02',
 'Design Number': 'AL87',
 'Engine Type': '624',
 'Engine Version': 'H12',
 'P': 24,
 'P_nom': 4495.0,
 'BMEP': 24.5,
 'serialNumber': '1486152',
 'id': 159397,
 'Count_OpHour': 1850.0,
 'val start': Timestamp('2022-01-10 00:00:00'),
 'oph@start': 54,
 'oph parts': 1796.0,
 'LOC': 0.043}

In [22]:
for item in dl[:]:
    erg = mp.historical_dataItem(e.id, item['id'], int(arrow.get('2023-03-19 18:15').timestamp()*1000), num_retries=0)
    if erg is not None:
        print(f"{item['name']:45} {str(erg):8} {str(item['unit']):8} {item['myPlantName']}")


ATD1                                          0.0      nan      1. ATD information for the table
ATD2                                          0.0      nan      2. ATD information for the table
Aux_CH4Cont                                   0.0      %        CH4 content
Aux_GasPress                                  0.0      mbar     Gas pressure
Aux_GasRail_rDPr_SetAct                       0.0      nan      Mainchamber gas train set-actual deviation
Aux_GasRail_rPr_Out_Calc                      -0.95    nan      Gas pressure gas train outlet - calculated
Aux_GasRail_rPr_a_Filt                        0.0      nan      Mainchamber gas train gas pressure after filter
Aux_GasRail_rPr_sp                            5.87     nan      Mainchamber gastrain set pressure
Aux_GasTemp                                   0.0      °C       Gas temperature
Aux_GasVol                                    930.0    m3/h     Gas volume
Aux_O2Cont                                    0.0      %        O2 content